### Important Definitions
**ID** - name of folder containing single patients dcm images <br>
**Slice** - slice number <br>
**ITSA** - interative threshold-seeking algorithm<br>
**ROI mask** - individual thigh after removing the subcutaneous fat and femur. Contains only muscle and inter/intra muscular fat<br>
**F1 fat segements** - segments of fat detected in the ROI mask after 1st round of ITSA (brightest fat segments)<br>
**F2 fat segements** - segments of fat detected in ROI mask after 2nd round of ITSA <br>
**ROI_MuscFatAreaPix** - number of pixels in ROI mask	<br>
**ROI_MuscFatAreaMM** - area of the ROI mask [mm^2]<br>
**ROI_MuscFatVolMM** - volume if the ROI mask [mm^3]	<br>
**F1_AreaPix** - number of pixels that correspond to the F1 fat segments in the ROI mask<br>
**F1_AreaMM** - area of the F1 fat segments in the ROI mask [mm^2]<br>
**F1_MuscAreaPix** - number of pixel remaining in ROI mask after removing F1 fat segments<br>
**F1_MuscAreaMM** - area of remaining ROI mask after removing F1 fat segments [mm^2]<br>
**F1_Perc** - percentage of F1 fat segments relative to entire ROI mask<br>
**F1_VolMM** - 	volume of the F1 fat segments in the ROI mask [mm^3]<br>
**F1_MuscVolMM** - volume of remaining ROI mask after removing F1 fat segments [mm^3]<br>
**F2_AreaPix** - number of pixels that correspond to the F2 fat segments in the ROI mask<br>
**F2_AreaMM** - area of the F2 fat segments in the ROI mask [mm^2]<br>
**Musc_noF2_AreaPix** - number of pixel remaining in ROI mask after removing F1 and F2 fat segments<br>	
**musc_no_F2_AreaMM** - area of remaining ROI mask after removing F1 and F2 fat segments [mm^2]<br>
**F1_Intensity** - mean value of F1 fat segments <br>	
**F2_Intensity** - mean value of F2 fat segments	<br>
**F2_cfactor** - correction of partial volume in pixels, percentage of the volume occupied by IMF within a partial volumed voxel (F2_Intensity/F1_Intensity)<br>
**F2_Perc_NOTc** - percentage of F2 fat segments relative to entire ROI mask without correcting for partial volume<br>
**F2_VolMM_NOTc** - volume of the F2 fat segments in the ROI mask without correcting for partial volume [mm^3]<br>
**BF_FatVol_NOTc** - sum of F1 and F2 fat segments volume without correcting for partial volume[mm^3]<br>
**F2_VolMM_c** - volume of the F2 fat segments in the ROI mask after correcting for partial volume[mm^3]<br>
**F2_MuscVolMMc** - volume of remaining ROI mask after removing F1 and F2 fat segments [mm^3]<br>
after correcting for partial volume<br>
**BF_FatVol_c** - volume from combining F1 and F2 fat segments after correcting for partial volume [mm^3]<br>
**F2_Perc_c** - percentage of F2 fat segments relative to entire ROI mask after correcting for partial volume<br>
**BF_Perc_c** - sum of F2_Perc and F1_Prec after correcting for partial volume<br>
**BF_AreaPix_NOTc** - number of pixels from combining F1 and F2 fat segment masks without correcting for partial volume<br>
**BF_AreaMM^2_NOTc** - area from combining F1 and F2 fat segment masks without correcting for partial volume [mm^2]<br>
**Musc_noBF_AreaPix_NOTc** - number of pixels in ROI mask after removing F1 and F2 fat segments without correcting for partial volume<br>	
**musc_no_BF_AreaMM_NOTc** - area of ROI mask after removing F1 and F2 fat segments without correcting for partial volume [mm^2]<br>
**BF_Perc_NOTc** - percentage of F1 and F2 fat segments relative to entire ROI mask without correcting for partial volume<br>
**BF_VolMM_NOTc** - volume from combining F1 and F2 fat segments without correcting for partial volume [mm^3]<br>
**Musc_noBF_VolMM_NOTc** -volume of ROI mask after removing F1 and F2 fat segments without correcting for partial volume [mm^3]<br>
**TOTAL_FatVol_c** - sum of all fat volumes from a patient (across 15 slices) after correcting for partial volume[mm^3]<br>
**TOTAL_MuscVol_c** - sum of all muscle volumes from a patient (across 15 slices) after correcting for partial volume[mm^3]<br>



### Image Processing Variables
#### Inhomogeneity Correction
Requires sharpening the image first the running inhomogeneity correction followed by a series of blurring, thresholding and denoising functions. The thighs are then isolated and their location noted. Another inhomogeneity correction occurs before the background is removed. The final image is the 2 thighs on a black background.
#### Within Snake
**alpha** - snakes energy, tendency to move away from original initiator (higher alpha=more freedom to move)-small here given that initiated snake is close to fascia <br>
**beta** - smoothness (higher beta = smoother) <br>
**w_line** -  attraction the dark/white (- means attracted to dark pixels, + to white pixels) <br>
**w_edge** - attraction to edges (higher=more attraction)  <br>
#### Island Removal
minsize = 8 

In [ ]:
'''
find *** to locate where changes need to be made
'''

In [ ]:
# Run definitions file
%run ITSA_definitions-Clean.ipynb

In [ ]:
#bring new imports
import shutil
import pydicom
from matplotlib import pyplot as plt

# Set image directory and output directory
patpath = r"D:\thighimages\"  #***change to directory with folders containing images to process

#determine total number of patients
totpat = len([d for d in os.listdir(patpath) if os.path.isdir(os.path.join(patpath, d))])

#opens first patient file to determine size
patdir = os.path.join(patpath, "9035449") #***change to first folder in directory that contains images to process
patdcm = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(patdir)
patsitk=sitk.ReadImage(patdcm)
read_data=sitk.GetArrayFromImage(patsitk) 

#create empty 4D array with size from number of patients and first patient file
shape = (totpat, read_data.shape[0], read_data.shape[1], read_data.shape[2]) 
pat4d = np.empty(shape, dtype='uint8') 

#start indexing
ID_count = 0
i = 0
for folder in os.listdir(patpath):
    print(folder)
    curr_folder_path = os.path.join(patpath, folder)
    for item in os.listdir(curr_folder_path):
        file_full_path = os.path.join(curr_folder_path, item)
        if item.lower().endswith((".nrrd", ".ds_store")): # skip outstanding files
            continue
        elif os.path.isdir(file_full_path):
            if not os.path.exists(folder_full_path):
                os.mkdir(folder_full_path)
            shutil.copytree(file_full_path, folder_full_path, dirs_exist_ok=True)
        else:
            ds = pydicom.dcmread(file_full_path)
            read_array = ds.pixel_array
            m = sharpen1(read_data=file_full_path, filename=item, mode="e_s")          
            m = brightness_correction(read_data=m[0], filename=m[1], mode="CLAHE", strength=6.2)
            m = blur(read_data=m[0], filename=m[1], mode="m_b", strength=5)
            m = blur(read_data=m[0], filename=m[1])
            m = blur(read_data=m[0], filename=m[1])
            m = blur(read_data=m[0], filename=m[1])
            m = blur(read_data=m[0], filename=m[1])
            m = blur(read_data=m[0], filename=m[1], mode="m_b",strength=7)
            m = thresholding(read_data=m[0], strength=55, filename=m[1])
            m = blur(read_data=m[0], filename=m[1], mode="m_b", strength=5)
            m = denoise(read_data=m[0], filename=m[1])
            m = blur(read_data=m[0], filename=m[1], mode="m_b", strength=11)
            m = fill_contours(read_data=m[0], filename=m[1], crop=True, dilation_iterations=6)
            m = blur(read_data=m[0], filename=m[1], mode="m_b", strength=13)

            f = brightness_correction(read_data=file_full_path,ds=ds, strength=2.3, filename=m[1])

            f = get_cleaned_image(read_data=f[0], mask=m[0], ds=ds, filename=f[1], show_images=False) 

            pat4d[ID_count, i, :, :] = f
           
            i += 1 #add to slice counting index
            total_pixels = m[0].size
            black_pixels = np.count_nonzero(m[0] == 0)
            white_pixels = np.count_nonzero(m[0] == 255)

            # Calculate the percentage
            black_percentage = (black_pixels / total_pixels) * 100
            white_percentage = (white_pixels / total_pixels) * 100

    ID_count += 1 
    i = 0 
        

In [ ]:
#left leg array
shape = (len(pat4d),len(pat4d[0]),len(pat4d[0,0]),len(pat4d[0,0]) )
lthigh_arr = np.empty(shape, dtype='uint8') 
for ID_count in range (len(pat4d)):
    print(ID_count)
    read_data = pat4d[ID_count,14] #check last slice as it is likely the biggest
    rmv_count,num_columns_to_remove = GetShift(read_data,'L')
    for i in range (len(pat4d[0])):
        read_data = pat4d[ID_count,i] 
        centered_image = LegSepNew(read_data,'L',rmv_count,num_columns_to_remove)
        new_centered_image = centered_image[:,:] 
        lthigh_arr[ID_count, i, :, :] = new_centered_image

In [ ]:
#right leg array
shape = (len(pat4d),len(pat4d[0]),len(pat4d[0,0]),len(pat4d[0,0]) ) 
rthigh_arr = np.empty(shape, dtype='uint8') 
for ID_count in range (len(pat4d)):
    print(ID_count)
    read_data = pat4d[ID_count,14] #check last slice as it is likely the biggest
    rmv_count,num_columns_to_remove = GetShift(read_data,'R')
    for i in range (len(pat4d[0])):
        read_data = pat4d[ID_count,i] 
        centered_image = LegSepNew(read_data,'R',rmv_count,num_columns_to_remove)
        new_centered_image = centered_image[:,:] 
        rthigh_arr[ID_count, i, :, :] = new_centered_image

In [ ]:
#check all images for cutoff errors (looking at 10 patients at a time)
j=15
for i in range(10):
    i = i + 0 #***change i to cycle through all images
    stackimages(rthigh_arr[i]) 

In [ ]:
# correcting any errors in thigh separation
def LegSepNew (image, side, rmv_count, num_columns_to_remove):
    read_data = image
    num_columns_to_remove = num_columns_to_remove
    rmv_count = rmv_count
    read_data = cv2.normalize(read_data, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    kernel = np.array([[0, 1, 0],
                [1, 1, 1],
                [0, 1, 0]], dtype = np.uint8)
    erode = cv2.erode(read_data, kernel, iterations=1)
    blur = cv2.medianBlur(erode, 3)
    blur = cv2.bilateralFilter(blur,9,10,50)                                               #locating 2 large bright regions (legs)
    ret, thresh = cv2.threshold(blur, 30, 255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    largest_contours = sorted([cv2.contourArea(obj) for obj in contours], reverse=True)[:2]
    thighs = [obj for obj in contours if cv2.contourArea(obj) in largest_contours]
    backtorgb = cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB)
    cv2.drawContours(backtorgb, [thighs[0]], -1, (255,0,0), 1)
    cv2.drawContours(backtorgb, [thighs[1]], -1, (0,0,255), 1)
    min_centroid = ()
    max_centroid = ()
    if side == 'L':
            for contour in thighs:
                # Calculate the centroid of the contour
                M = cv2.moments(contour)
                cX = int(M['m10'] / M['m00'])
                cY = int(M['m01'] / M['m00'])
                if len(min_centroid) == 0 or cX < min_centroid[0]:     #chooses centroid with smaller x value (i.e. left side)
                    min_centroid = (cX, cY, contour)
            min_contour = min_centroid[2]
            end_x = min_contour[:, :, 0].max() # Find the ending x-coordinate (rightmost point) of the contour
            #check if columns were removed in sample slice
            end_x=251 #***change to x value that matches a slice from same patient that did not have an error
            if rmv_count > 0:
                centered_image = read_data[:,num_columns_to_remove:end_x]
                start_x = min_contour[:, :, 0].min()
                
                if start_x < num_columns_to_remove:
                    print("using backup")
                    centered_image = BackupLegSeparation(read_data, "L")
            else:
                centered_image = read_data[:, :end_x] #cut off at centroid
            
            #reform new shape
            desired_shape = (256, 256)  # define desired shape (rows, columns)
            np.expand_dims(centered_image,axis=2)
            if desired_shape[1] - centered_image.shape[1] > 0: #add columns
                num_columns_to_add = desired_shape[1] - centered_image.shape[1]
                centered_image_f = np.pad(centered_image, ((0, 0), (0, num_columns_to_add)), mode='constant')
            elif desired_shape[1] - centered_image.shape[1] < 0: #remove columns
                num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
                centered_image_f = centered_image[:,:desired_shape[1]]
                print("Too Big")
            else:
                centered_image_f = centered_image              
                         
                
    if side == 'R':
        max_cX = None
        for contour in thighs:
            # Calculate the centroid of the contour
            M = cv2.moments(contour)
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
            if max_cX is None or cX > max_cX:
                max_cX = cX  # Update max_cX with the current cX
                max_centroid = (cX, cY, contour) #pick right side

        max_contour = max_centroid[2]
        start_x = max_contour[:, :, 0].min()# Find the ending x-coordinate (leftmost point) of the contour
        desired_shape = (256, 256)  # define desired shape (rows, columns)
        start_x=259 #***change to x value that matches a slice from same patient that did not have an error
        #check if columns were removed in sample slice
        if rmv_count > 0:
            cutoff = 512 - num_columns_to_remove
            centered_image = read_data[:,start_x:cutoff]
            start_x = max_contour[:, :, 0].max()
        else:
            centered_image = read_data[:, start_x:]
        
        
        if desired_shape[1] - centered_image.shape[1] > 0: #add columns
            num_columns_to_add = desired_shape[1] - centered_image.shape[1]
            centered_image_f = np.pad(centered_image, ((0, 0), (num_columns_to_add,0)), mode='constant')
        elif desired_shape[1] - centered_image.shape[1] < 0: #remove columns
            num_columns_to_remove = centered_image.shape[1] - desired_shape[1]
            centered_image_f = centered_image[:, num_columns_to_remove:]
        else:
            centered_image_f = centered_image
            
    return centered_image_f

In [ ]:
#fixing any errors cutoffs
ID_count = 37 #***change to patient index that had error in thigh separation
i = 14 #***change to slice index that had error in thigh separation

#confirm correct image was selected
plt.imshow(rthigh_arr[ID_count][i]) #***adjust lthigh_arr and rthigh_arr depending on which thigh cut offs are being fixed
plt.show()

read_data = pat4d[ID_count,14]
rmv_count,num_columns_to_remove = GetShift(read_data,'R') #***adjust L and R depending on which thigh cut offs are being fixed
read_data = pat4d[ID_count,i] 
centered_image = LegSepNew(read_data,'R',rmv_count,num_columns_to_remove) #***adjust L and R depending on which thigh cut offs are being fixed
new_centered_image = centered_image[:,:] #centered image is 3D array need to just take usable parts
rthigh_arr[ID_count, i, :, :] = new_centered_image #***adjust lthigh_arr and rthigh_arr depending on which thigh cut offs are being fixed

#visualize corrected image
plt.imshow(rthigh_arr[ID_count][i])#***adjust lthigh_arr and rthigh_arr depending on which thigh cut offs are being fixed
plt.show()

In [ ]:
I8c_subcfat_imp_arr=[]
I8c_subcfat_imp_arr=rthigh_arr #***adjust lthigh_arr and rthigh_arr depending on which side is being processed

I8c_roi_imp_arr=[]
I8c_roi_imp_arr=rthigh_arr #***adjust lthigh_arr and rthigh_arr depending on which side is being processed

In [ ]:
for ID_count,files in enumerate(os.listdir(patpath)):
        print(f"{ID_count}.{files}") 

In [ ]:
#create masks only
j=15

Ivol8_OG=[]
Ivol8_L=[]
Ivol8c_subcfat_L=[]  
Ivol8c_roi_L=[]
Ivol8c_c_s_L=[]
subcfat_ring_L=[]
musc_mask_final_L=[]
t_L=[]
musc_mask2_L=[] 
snake1_mask_L=[]
bone_z_L=[]
bonem_final_L=[]
bonem_c_L=[]
roi_mask_L=[]
fatseg_mask_final_R1_L=[]
fatseg_final_R1_L=[]
fatseg_mask_final_R2_L=[]
fatseg_final_R2_L=[]
for_zcheck_S2R1_L=[]
fatseg1_mask_final_S1_L=[]
fatseg1_final_S1_L=[]
fatseg2_mask_final_S1_L=[]
fatseg2_final_S1_L=[]
final_fat_mask_L=[]
final_fat_L=[]


for ID_count, files in enumerate(os.listdir(patpath)):
    #GET IMAGES
    image_path = os.path.join(patpath,files)
    slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(image_path)
    image_OG = sitk.ReadImage(slice_filenames)
    Ivol8_OG,j_OG,plotx=getimages(image_OG) 
    
    #APPLY START SLICE
    Ivol8c_subcfat=I8c_subcfat_imp_arr[ID_count]
    Ivol8c_roi=I8c_roi_imp_arr[ID_count]
    Ivol8=Ivol8_OG

    #Extend ALL four sides by 40 pixels 
    Ivol8,pixels=enlarge_size(Ivol8,40)
    Ivol8c_subcfat,pixels=enlarge_size(Ivol8c_subcfat,40)
    Ivol8c_roi,pixels=enlarge_size(Ivol8c_roi,40)

    Ivol8_L.append([])
    Ivol8_L[ID_count]=Ivol8

    Ivol8c_roi_L.append([])
    Ivol8c_roi_L[ID_count]=Ivol8c_roi

    Ivol8c_subcfat_L.append([])
    Ivol8c_subcfat_L[ID_count]=Ivol8c_subcfat
    try:
        #MUSCLE MASK
        subcfat=get_subcfatvol(Ivol8c_subcfat)
        musc_fat_mask=get_musc_fat_mask(Ivol8c_subcfat)
        subcfat_ring=get_subcfat_ring(musc_fat_mask)

        curvature_flow=apply_CurvatureFilter(Ivol8c_subcfat*musc_fat_mask,Ivol8c_roi) 
        Ivol8c_c_s=enhance_sharpness(curvature_flow)
        subcfatvol=get_subcfatvol(Ivol8c_c_s) 

        subcfatvol_augmented=subcfatvol+subcfat_ring 
        th, subcfatvol_augmented = cv2.threshold(subcfatvol_augmented, 0, 1, cv2.THRESH_BINARY)

        #--for calf:
        subcfatvol_augmented = np.uint8(morphology.remove_small_holes(label(subcfatvol_augmented),area_threshold=300, connectivity=1))
        th, subcfatvol_augmented = cv2.threshold(subcfatvol_augmented, 0, 1, cv2.THRESH_BINARY)



        t, musc_mask2=darkpieces(subcfatvol_augmented)


        overshoot1,overshoot_removed1=remove_subcfat_overshoot(musc_mask2,Ivol8c_c_s,1)
        overshoot2,overshoot_removed2=remove_subcfat_overshoot(overshoot_removed1,Ivol8c_c_s,2)
        b=keep_overlaps(overshoot_removed2,1)
        musc_mask2=b


        mask_contours, contour_coords_L=get_contours(musc_mask2)
        hull, hull_demonstration,hull_coords=convex_hull(contour_coords_L)
        hullmask=floodfillall(hull) 

        hullmask_contour, hull_coords2=get_contours(hullmask)


        initiator_coords=compatible_coordlist(hull_coords2) 

        #REMOVE VESSELS
        vessels_removed_mask, vessels_present=remove_vessels(Ivol8c_c_s)
        pre_vessels_mask=vessels_removed_mask-vessels_present
        vessels_mask=get_vessels()

        whites=get_whites() #generate white mask for snakes to follow 
        snake_im=get_snakeim()
        bilateral_t=bilat_fil_snake_im(snake_im)
        snake1_coords=primary_snake()
        snake1_mask=snaketomask(snake1_coords)


        snake1_mask_L.append([])
        snake1_mask_L[ID_count]=snake1_mask


        #BONE REMOVAL
        musc_mask_final=snake1_mask
        bone_prep=get_boneprep(Ivol8c_subcfat)
        bone_prep2=find_potentialbone(bone_prep)


        bone_z,bone_nonz=Z_connectivity_w_adj(bone_prep2)

        bone_z_L.append([])
        bone_z_L[ID_count]=bone_z


        bonem_final=find_potentialbone2(bone_z) 
        bonem_final_L.append([])
        bonem_final_L[ID_count]=bonem_final



        cortical=corticaloutline(Ivol8c_c_s)
        bonem_c=merge_bones(cortical,bonem_final)

        bonem_c_L.append([])
        bonem_c_L[ID_count]=bonem_c

        roi_mask, roi=get_roi(snake1_mask,bonem_c,Ivol8c_subcfat)  

        roi_mask_L.append([])
        roi_mask_L[ID_count]=roi_mask
        print(f"{ID_count}. {files} success")

    except: #skip any errors
        print(f"{ID_count}. {files} encountered an error")
        #create placeholders
        snake1_mask_L.append(None)
        roi_mask_L.append(None)
        bonem_c_L.append(None)
        bonem_final_L.append(None)
        bone_z_L.append(None)
        for_zcheck_S2R1_L.append(None)
        fatseg1_mask_final_S1_L.append(None)
        fatseg1_final_S1_L.append(None)
        fatseg2_mask_final_S1_L.append(None)
        fatseg2_final_S1_L.append(None)
        final_fat_mask_L.append(None)
        final_fat_L.append(None)
        fatseg_mask_final_R1_L.append(None)
        fatseg_final_R1_L.append(None)
        fatseg_mask_final_R2_L.append(None)
        fatseg_final_R2_L.append(None)
        continue


In [ ]:
#run through all ROIs and classify quality
data = pd.DataFrame(columns=['ID','slice','Button Clicked'])
for ID_count,files in enumerate(os.listdir(patpath)):
    if ID_count != 58: #***change to skip any patients that were not able to generate a mask (i.e. error in previous step)
        quality_check(Ivol8c_roi_L[ID_count],roi_mask_L[ID_count],f"{ID_count}. {files}")
    else:
        continue

In [ ]:
#cycle through entire dataframe
output_dir = r"D:\thighimages_tags\" #***change to output directory for tags to be fixed using Sliceomatic
redo = []
for i in range(len(data)):
    ID = str(data['ID'][i])
    ID_count = int(ID.split('.')[0]) #get patient index
    slic = data['slice'][i].astype(int) - 1 # get slice index
    button = str(data['Button Clicked'][i]) # determine which button applies
    if button == 'Good': #skip patient with good ROI
        continue
    if button == 'Overshoot' or button == 'Undershoot':
        #create original dicom
        img = sitk.GetImageFromArray(Ivol8c_roi_L[ID_count][slic]) 
        output_filename=os.path.join(output_dir, f'{ID}_Slice{slic}.dcm') 
        sitk.WriteImage(img, output_filename) 
        print(f"{ID}.{slic} Image success")
        #create tags
        stid = ID.split('.')[1]
        volsitk = sitk.GetImageFromArray(Ivol8c_roi_L[ID_count][slic]) #sitk of original image
        imfbonemsc = roi_mask_L[ID_count][slic]#numpy array of roi
        slc = slic #identify the slice number but ones isnt 3d
        writetag()
        root, root2 = writetofolder(output_filename)
        print(f"{ID}.{slic} ROI success")
    if button == 'Incorrect Thigh Cut':
        redo.append((ID,slic)) #add patiient file name and slice number to redo list to be redone later


In [ ]:
#run through ITSA with corrected tags
path = r"D:\thighimages\" #***change to directory with folders containing images to process
tag_path = r"D:\thighimages_tagscorrected\" #***change to directory with folders containing edited tags
Ivol8_OG=[]
Ivol8_L=[]
Ivol8c_subcfat_L=[]  
Ivol8c_roi_L=[]
Ivol8c_c_s_L=[]
subcfat_ring_L=[]
musc_mask_final_L=[]
t_L=[]
musc_mask2_L=[]
snake1_mask_L=[]
bone_z_L=[]
bonem_final_L=[]
bonem_c_L=[]
roi_mask_L=[]
fatseg_mask_final_R1_L=[]
fatseg_final_R1_L=[]
fatseg_mask_final_R2_L=[]
fatseg_final_R2_L=[]
for_zcheck_S2R1_L=[]
fatseg1_mask_final_S1_L=[]
fatseg1_final_S1_L=[]
fatseg2_mask_final_S1_L=[]
fatseg2_final_S1_L=[]
final_fat_mask_L=[]
final_fat_L=[]
shape = (15,336,336)
roi_tag = np.empty(shape, dtype='uint8') 
roi = np.empty(shape, dtype='uint8')
img_hold = np.empty(shape, dtype='uint8')
count = 0
slcs = []
shape2 = (3,15,336,336)
roi_mask_L = []


for ID_count, files in enumerate(os.listdir(path)):
    #GET IMAGES
    image_path = os.path.join(path,files)
    slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(image_path)
    image_OG = sitk.ReadImage(slice_filenames)


    Ivol8_OG,j_OG,plotx=getimages(image_OG) #get all images
    
    #APPLY START SLICE
    Ivol8c_subcfat=I8c_subcfat_imp_arr[ID_count]
    Ivol8c_roi=I8c_roi_imp_arr[ID_count]
    Ivol8=Ivol8_OG

    #Extend ALL four sides by 40 pixels 
    Ivol8,pixels=enlarge_size(Ivol8,40)
    Ivol8c_subcfat,pixels=enlarge_size(Ivol8c_subcfat,40)
    Ivol8c_roi,pixels=enlarge_size(Ivol8c_roi,40)

    Ivol8_L.append([])
    Ivol8_L[ID_count]=Ivol8

    Ivol8c_roi_L.append([])
    Ivol8c_roi_L[ID_count]=Ivol8c_roi

    Ivol8c_subcfat_L.append([])
    Ivol8c_subcfat_L[ID_count]=Ivol8c_subcfat
    if ID_count == 54: #***change to skip any patients that were not able to generate a mask (i.e. error in previous step)
        print(f"{ID_count}. {files} encountered an error")
        #create placeholders
        snake1_mask_L.append(None)
        roi_mask_L.append(None)
        bonem_c_L.append(None)
        bonem_final_L.append(None)
        bone_z_L.append(None)
        for_zcheck_S2R1_L.append(None)
        fatseg1_mask_final_S1_L.append(None)
        fatseg1_final_S1_L.append(None)
        fatseg2_mask_final_S1_L.append(None)
        fatseg2_final_S1_L.append(None)
        final_fat_mask_L.append(None)
        final_fat_L.append(None)
        fatseg_mask_final_R1_L.append(None)
        fatseg_final_R1_L.append(None)
        fatseg_mask_final_R2_L.append(None)
        fatseg_final_R2_L.append(None)

    else:
        #MUSCLE MASK
        subcfat=get_subcfatvol(Ivol8c_subcfat)
        musc_fat_mask=get_musc_fat_mask(Ivol8c_subcfat)
        subcfat_ring=get_subcfat_ring(musc_fat_mask)

        curvature_flow=apply_CurvatureFilter(Ivol8c_subcfat*musc_fat_mask,Ivol8c_roi) 
        Ivol8c_c_s=enhance_sharpness(curvature_flow)
        subcfatvol=get_subcfatvol(Ivol8c_c_s) 

        subcfatvol_augmented=subcfatvol+subcfat_ring 
        th, subcfatvol_augmented = cv2.threshold(subcfatvol_augmented, 0, 1, cv2.THRESH_BINARY)

        #--for calf:
        subcfatvol_augmented = np.uint8(morphology.remove_small_holes(label(subcfatvol_augmented),area_threshold=300, connectivity=1))
        th, subcfatvol_augmented = cv2.threshold(subcfatvol_augmented, 0, 1, cv2.THRESH_BINARY)

        t, musc_mask2=darkpieces(subcfatvol_augmented)


        overshoot1,overshoot_removed1=remove_subcfat_overshoot(musc_mask2,Ivol8c_c_s,1)
        overshoot2,overshoot_removed2=remove_subcfat_overshoot(overshoot_removed1,Ivol8c_c_s,2)
        b=keep_overlaps(overshoot_removed2,1)
        musc_mask2=b


        mask_contours, contour_coords_L=get_contours(musc_mask2)
        hull, hull_demonstration,hull_coords=convex_hull(contour_coords_L)
        hullmask=floodfillall(hull) 

        hullmask_contour, hull_coords2=get_contours(hullmask)


        initiator_coords=compatible_coordlist(hull_coords2) 
        
        #REMOVE VESSELS
        vessels_removed_mask, vessels_present=remove_vessels(Ivol8c_c_s)
        pre_vessels_mask=vessels_removed_mask-vessels_present
        vessels_mask=get_vessels()

        whites=get_whites() #generate white mask for snakes to follow 
        snake_im=get_snakeim()
        bilateral_t=bilat_fil_snake_im(snake_im)
        snake1_coords=primary_snake()
        snake1_mask=snaketomask(snake1_coords)


        snake1_mask_L.append([])
        snake1_mask_L[ID_count]=snake1_mask


        #BONE REMOVAL
        musc_mask_final=snake1_mask
        bone_prep=get_boneprep(Ivol8c_subcfat)
        bone_prep2=find_potentialbone(bone_prep)


        bone_z,bone_nonz=Z_connectivity_w_adj(bone_prep2)

        bone_z_L.append([])
        bone_z_L[ID_count]=bone_z


        bonem_final=find_potentialbone2(bone_z) 
        bonem_final_L.append([])
        bonem_final_L[ID_count]=bonem_final


        cortical=corticaloutline(Ivol8c_c_s)
        bonem_c=merge_bones(cortical,bonem_final)

        bonem_c_L.append([])
        bonem_c_L[ID_count]=bonem_c

        roi_mask, roi=get_roi(snake1_mask,bonem_c,Ivol8c_subcfat)  
        roi_mask_L.append([])
        roi_mask_L[ID_count]=roi_mask

    #look for edited tags and update ROI here
        checking_file = os.path.join(tag_path, files)
        # Check if the file has any associated tags
        if os.path.exists(checking_file):
            folder = os.path.join(checking_file,"OutputTags")
            tagarray, tagsln, xdim, ydim  = tagtoarray(folder) #get all tags from folder
            for filename in os.listdir(folder):
                # Extract slice number
                slpos = filename.find(".dcm.tag")
                if slpos < 1: #does not have .dcm.tag and therefore is regular image
                    image = getOG(os.path.join(folder,filename))
                    img_hold[count] = image
                elif slpos > 1 and filename[slpos-1].isdigit():
                    # Collect all consecutive digits preceding ".dcm.tag"
                    digit_str = ""
                    index = slpos - 1

                    while index >= 0 and filename[index].isdigit():
                        digit_str = filename[index] + digit_str
                        index -= 1
                    slicenum = int(digit_str.replace("_", "")) #gives corresponding slice number
                    slcs.append(slicenum)
                    count +=1 
            #goes through all slices and assigns roi based on slice number
            for k in range(count):
                roi_tag[k] = tagarray[k]
                q = slcs[k]
                roi[q] = roi_tag[k] * img_hold[k]
                roi_mask_L[ID_count][q]=roi_tag[k]
            count = 0 #reset counter

        #========ITSA ========#
        #Set 1: Round 1:
        initial_th_S1R1=initial_th(roi)
        th_S1R1,fatseg_mask_S1R1,fatseg_S1R1=apply_ITSA_no_Z(roi,initial_th_S1R1,8) 
        #Set 1: Round 2:
        roi_for_S1R2=subtract_fat1(fatseg_S1R1)
        initial_th_S1R2=initial_th(roi_for_S1R2)
        th_S1R2,fatseg_mask_S1R2,fatseg_S1R2=apply_ITSA_no_Z(roi_for_S1R2,initial_th_S1R2,8) 
    

        #Set 2 prep?:
        for_zcheck_S2R1=apply_th(th_S1R2,roi) 
        fatseg1_mask_final_S1,fatseg1_final_S1,fatseg2_mask_final_S1,fatseg2_final_S1,=get_final_fatsegs_S1(for_zcheck_S2R1*roi,th_S1R1,for_zcheck_S2R1,roi) 

        #Set 2: Round 1:
        th_S2R1,fatseg_mask_S2R1,fatseg_S2R1=apply_ITSA_w_Z(roi,initial_th_S1R1,for_zcheck_S2R1,8) 
        
        #Set 2: Round 2:
        roi_for_S2R2=subtract_fat1(fatseg_S2R1) 
        initial_th_S2R2=initial_th(roi_for_S2R2)
        th_S2R2,fatseg_mask_S2R2,fatseg_S2R2=apply_ITSA_w_Z(roi_for_S2R2,initial_th_S2R2,for_zcheck_S2R1,8) 
        
        #Final connectivity check:
        for_final_zcheck=apply_th(th_S2R2,roi) 
        final_fat_mask, final_fat=apply_final_zcheck(th_S2R2,for_final_zcheck,8) 
        fatseg_mask_final_R1,fatseg_final_R1,fatseg_mask_final_R2,fatseg_final_R2=get_final_fatsegs(final_fat,th_S2R1,final_fat_mask,roi)

        #from set1:

        for_zcheck_S2R1_L.append([])
        fatseg1_mask_final_S1_L.append([])
        fatseg1_final_S1_L.append([])
        fatseg2_mask_final_S1_L.append([])
        fatseg2_final_S1_L.append([])

        for_zcheck_S2R1_L[ID_count]=for_zcheck_S2R1
        fatseg1_mask_final_S1_L[ID_count]=fatseg1_mask_final_S1
        fatseg1_final_S1_L[ID_count]=fatseg1_final_S1
        fatseg2_mask_final_S1_L[ID_count]=fatseg2_mask_final_S1
        fatseg2_final_S1_L[ID_count]=fatseg2_final_S1

        #from set 2:
        final_fat_mask_L.append([])
        final_fat_L.append([])

        fatseg_mask_final_R1_L.append([])
        fatseg_final_R1_L.append([])
        fatseg_mask_final_R2_L.append([])
        fatseg_final_R2_L.append([])

        final_fat_mask_L[ID_count]=final_fat_mask
        final_fat_L[ID_count]=final_fat

        fatseg_mask_final_R1_L[ID_count]=fatseg_mask_final_R1
        fatseg_final_R1_L[ID_count]=fatseg_final_R1

        fatseg_mask_final_R2_L[ID_count]=fatseg_mask_final_R2
        fatseg_final_R2_L[ID_count]=fatseg_final_R2

        print(f"{ID_count}. {files} success")



In [ ]:
im_spacing = image_OG.GetSpacing() #get spacing for original image - for calculations later
ID=files
results_df=[]
for ID_count,files in enumerate(os.listdir(patpath)):
        try:
            ID=files
            results_df.append([])    

            final_F1_mask=fatseg_mask_final_R1_L[ID_count] 
            final_F1=fatseg_final_R1_L[ID_count]

            final_F2_mask=fatseg_mask_final_R2_L[ID_count]
            final_F2=fatseg_final_R2_L[ID_count]

            final_BF_mask=final_fat_mask_L[ID_count]
            final_BF=final_fat_L[ID_count]

            final_roi_mask=roi_mask_L[ID_count]
            final_roi=roi_mask_L[ID_count]*Ivol8c_roi_L[ID_count] 


            results_df[ID_count]=calc2(final_F1_mask,final_F1,final_F2_mask,final_F2,final_BF_mask,final_BF,final_roi_mask,final_roi)
        except:
            print(f"{files} failed")
            continue



In [ ]:
store_dir=r"thighimage_IMFoutputs.xlsx" #***change to excel file name

for ID_count,files in enumerate(os.listdir(path)): 
        try:
            if ID_count==0:#if first ID

                with pd.ExcelWriter(store_dir) as writer:  
                    results_df[ID_count].to_excel(writer, sheet_name='Results',index=False)    

            else: #append to same SHEET (not just excel)     
                results_to_append=append_df_to_sheet(results_df[ID_count],store_dir,'Results') #read sheet name "Results" and append new results_table
                with pd.ExcelWriter(store_dir) as writer:  
                    results_to_append.to_excel(writer, sheet_name='Results',index=False) 

        except:
            print(f"{files} export failed")
            # Load the existing Excel file into a DataFrame
            df_excel = pd.read_excel(store_dir, sheet_name='Results')
            # Create a new DataFrame with the data to append
            data_to_append = pd.DataFrame({'Filename': [files], 'Error': ['Patient Mask Not Complete']})
            # Concatenate the two DataFrames vertically
            result = pd.concat([df_excel, data_to_append], ignore_index=True)
            # Save the concatenated DataFrame back to the Excel file without index values
            with pd.ExcelWriter(store_dir, engine='openpyxl') as writer:
                result.to_excel(writer, sheet_name='Results', index=False)
        else:
            print(f"{files} export success")
